# Exploratory data analysis (EDA) and Big Tabular Data
# Ben

In [13]:

"""
Demo about exploratory data analysis --manipulation-  using 
pandas, matplotlib, seaborn, numpy on a  
Jupyter Notebooks
Ben bencouver@gmail.com
Inspired on several courses and tutorials, Specially techniques
learned from Roland Jeannier on pandas optimization and big tabular data
keeps growing
Sorry about mixing english and Spanish
"""
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns #for pair plots
import datetime
%matplotlib inline
plt.style.use('fivethirtyeight')

"""Ingest dataframe"""

#Load the Titanic Dataset 
#(Cant automate download cuz it requires login to Kaggle, maybe using the Kaggle API?)

features1 = pd.read_csv('/home/benjamin/Downloads/train.csv')
features1.head(5)



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
features2 = pd.read_csv('/home/benjamin/Downloads/test.csv')
features2.head(5)



,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [30]:
#Concatenates pd objects along some axis, in this case is like a paste along the horizontal
#Note that it ommits columns not present in the second dataframe
features3= pd.concat([features1,features2], axis=0)
features3.head(5)


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [45]:
def basic_exploration(df):
    dict_list = []
    for col in df.columns:
        data = df[col]
        dict_ = {}
        # How many unique values per column
        dict_.update({"unique_count" : len(data.unique())})
        """How many nulls per column?"""  
        dict_.update({"null_count" : data.isnull().sum()})
        # Types of our data
        dict_.update({"data_type" : set([type(d).__name__ for d in data])})
        dict_list.append(dict_)
    new_df = pd.DataFrame(dict_list)
    new_df.index = df.columns
    new_df.sort_values(by=['null_count', 'unique_count'], ascending=[True, False], inplace=True)


    return new_df

basic_exploration(features3)

,data_type,null_count,unique_count
PassengerId,{int},0,1309
Name,{str},0,1307
Ticket,{str},0,929
Parch,{int},0,8
SibSp,{int},0,7
Pclass,{int},0,3
Sex,{str},0,2
Fare,{float},1,282
Embarked,"{float, str}",2,4
Age,{float},177,100


 The following are experiments on the performance of replacing null values on the Age column by a -1, 
 plus how to check he memory fingerprint of a dataframe

In [46]:


def test_iterrow(df):
    for (i, row) in df.iterrows():
        val = row['Age']
        if math.isnan(val):
                row['Age'] = -1
%timeit test_iterrow(features1)

10 loops, best of 3: 66.3 ms per loop


In [47]:

def test_get_value(df):
    for i in df.index:
        val = df.get_value(i,'Age')
        if math.isnan(val):
            df.set_value( i,'Age',-1)
%timeit test_get_value(features1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


100 loops, best of 3: 4.68 ms per loop


In [49]:
def test_loc(df):
    for i in df.index:
        val = df.loc[i,'Age']
        if math.isnan(val):
            df.loc[i,'Age'] = -1
%timeit test_loc(features1)



100 loops, best of 3: 9.75 ms per loop


In [50]:
features1.shape


(891, 12)

In [51]:
features2.shape

(418, 11)

In [52]:
features3.shape

(1309, 12)

Check how the index "type" changes from RangeIndex in feature1 df to Int64Index after a 
concatenation or aan append of dataframes, why? (this affects applying the above ops to the 
concatenated/appended dfs)

In [53]:
features3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1132 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 172.9+ KB


In [54]:
features2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [55]:
features4 = features1.append(features2)
features4.head(5)


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [56]:
features4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [57]:
features3.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1132 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 448.3 KB
